Zuerst erstellen wir ein Dataset.


In [1]:
class Animal:
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description

    def __repr__(self):
        return f"Animal(name='{self.name}', description='{self.description}')"


# Liste der Tiere mit deutschen Beschreibungen
animals = [
    Animal(
        "Löwe",
        "Der Löwe ist ein großes, kräftiges Raubtier und bekannt als König der Tiere. Er lebt hauptsächlich in Afrika und jagt in Rudeln. Männliche Löwen haben eine markante Mähne. Sie sind für ihre Stärke und ihr lautes Brüllen bekannt.",
    ),
    Animal(
        "Elefant",
        "Der Elefant ist das größte Landtier der Welt und lebt in Afrika und Asien. Er hat große Ohren, einen langen Rüssel und dicke Beine. Elefanten sind für ihre Intelligenz und sozialen Bindungen bekannt. Sie leben oft in Herden und kommunizieren über lange Distanzen.",
    ),
    Animal(
        "Panda",
        "Der Panda ist ein großes, schwarz-weißes Säugetier, das hauptsächlich Bambus frisst. Er stammt aus China und ist ein Symbol für den Artenschutz. Pandas sind ruhig und verbringen die meiste Zeit mit der Nahrungssuche. Sie sind eine bedrohte Tierart.",
    ),
    Animal(
        "Wolf",
        "Der Wolf ist ein soziales Raubtier, das in Rudeln lebt und ein Vorfahre des Haushundes ist. Wölfe sind intelligente Jäger und kommunizieren über Heulen. Sie kommen in Nordamerika, Europa und Asien vor. Sie haben ein starkes Gemeinschaftsgefühl.",
    ),
    Animal(
        "Känguru",
        "Das Känguru ist ein Beuteltier, das in Australien heimisch ist und sich hüpfend fortbewegt. Es hat kräftige Hinterbeine und einen langen Schwanz. Kängurus leben oft in Gruppen und tragen ihre Jungen im Beutel. Sie sind Pflanzenfresser und an trockenes Klima angepasst.",
    ),
]

# Ausgabe der Liste
print(animals)

[Animal(name='Löwe', description='Der Löwe ist ein großes, kräftiges Raubtier und bekannt als König der Tiere. Er lebt hauptsächlich in Afrika und jagt in Rudeln. Männliche Löwen haben eine markante Mähne. Sie sind für ihre Stärke und ihr lautes Brüllen bekannt.'), Animal(name='Elefant', description='Der Elefant ist das größte Landtier der Welt und lebt in Afrika und Asien. Er hat große Ohren, einen langen Rüssel und dicke Beine. Elefanten sind für ihre Intelligenz und sozialen Bindungen bekannt. Sie leben oft in Herden und kommunizieren über lange Distanzen.'), Animal(name='Panda', description='Der Panda ist ein großes, schwarz-weißes Säugetier, das hauptsächlich Bambus frisst. Er stammt aus China und ist ein Symbol für den Artenschutz. Pandas sind ruhig und verbringen die meiste Zeit mit der Nahrungssuche. Sie sind eine bedrohte Tierart.'), Animal(name='Wolf', description='Der Wolf ist ein soziales Raubtier, das in Rudeln lebt und ein Vorfahre des Haushundes ist. Wölfe sind intellige

Und laden Examples in das Dataset hoch.


In [2]:
from langsmith import Client

client = Client()

dataset_name = "animals"
if not client.has_dataset(dataset_name=dataset_name):
    client.create_dataset(dataset_name=dataset_name)


client.create_examples(
    dataset_name=dataset_name,
    inputs=animals,
)

Als nächstes erstellen wir uns eine Demo chain zum Übersetzen der Texte.


In [3]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from anthropic import RateLimitError as AnthropicRateLimitError

configurable_model = init_chat_model(
    model="gpt-4o-mini",
    temperature=0,
    configurable_fields=("model", "model_provider", "temperature", "max_tokens"),
)

prompt = PromptTemplate.from_template("""
Übersetze die folgenden deutschen Beschreibungen von Tieren ins Englische und füge dabei keine Kommentare hinzu:
                                      
{description}
                                                                                                                               
""")

configurable_chain = prompt | configurable_model | (lambda x: {"animal": x.content})


async def oai_experiment(input: dict) -> dict:
    return await configurable_chain.ainvoke(input)


async def anthropic_experiment(input: dict) -> dict:
    return (
        await configurable_chain.with_config(model="claude-3-5-sonnet-20240620")
        .with_retry(
            retry_if_exception_type=(
                AnthropicRateLimitError,
            ),  # Careful, must be a Tuple
            wait_exponential_jitter=True,
            stop_after_attempt=10,
        )
        .ainvoke(input)
    )

/var/folders/01/ywnzdvn56c16b5vdy9c7_q8w0000gn/T/ipykernel_31559/2706591685.py:5: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  configurable_model = init_chat_model(


Jetzt führen wir die Experimente aus und speichern die Ergebnisse.


In [4]:
from langsmith import aevaluate

anthropic_experiment_results = await aevaluate(
    anthropic_experiment,
    data=dataset_name,
    experiment_prefix="sonnet-3.5",
)
print(f"anthropic experiment name: {anthropic_experiment_results.experiment_name}")
oai_experiment_results = await aevaluate(
    oai_experiment,
    data=dataset_name,
    experiment_prefix="gpt-4o",
)
print(f"oai experiment name: {oai_experiment_results.experiment_name}")
experiment_names = [
    anthropic_experiment_results.experiment_name,
    oai_experiment_results.experiment_name,
]

View the evaluation results for experiment: 'sonnet-3.5-c4241dda' at:
https://smith.langchain.com/o/eb86f40f-e90c-4eb8-846e-94f09f6daf2e/datasets/cdf21ef4-42ea-4e5d-ae2e-8972df7ed2d6/compare?selectedSessions=9122d1a2-1b61-4b7a-a9fd-f364411b9238




0it [00:00, ?it/s]

anthropic experiment name: sonnet-3.5-c4241dda
View the evaluation results for experiment: 'gpt-4o-e5ca343e' at:
https://smith.langchain.com/o/eb86f40f-e90c-4eb8-846e-94f09f6daf2e/datasets/cdf21ef4-42ea-4e5d-ae2e-8972df7ed2d6/compare?selectedSessions=03a4c7e1-d042-4963-9cee-e3ca7aef8801




0it [00:00, ?it/s]

oai experiment name: gpt-4o-e5ca343e


In [5]:
import uuid
from langsmith.schemas import ComparativeExperiment

comparative_experiment_id = uuid.uuid4()
comparative_experiment_name_suffix = "testcomparison"
comparate_experiment_name = (
    "-vs-".join(experiment_names) + "-" + comparative_experiment_name_suffix
)
experiments = [client.read_project(project_name=exp) for exp in experiment_names]
experiment_ids = [str(exp.id) for exp in experiments]
ref_datasets_ = [str(exp.reference_dataset_id) for exp in experiments]
if not len(set(ref_datasets_)) == 1:
    raise ValueError("All experiments must have the same reference dataset.")

dataset_id = client.read_dataset(dataset_name=dataset_name).id

response = client.request_with_retries("GET", f"/datasets/{dataset_id}/comparative")
comp_exp = None
for exp in response.json():
    if exp["name"] == comparate_experiment_name:
        print(f"comparative experiment already exists: {exp['name']}")
        comp_exp = ComparativeExperiment(**exp)
        break
if comp_exp is None:
    comp_exp = client.create_comparative_experiment(
        name=comparate_experiment_name,
        experiments=experiment_ids,
        id=comparative_experiment_id,
    )
    print(f"comparative experiment created: {comp_exp.name}")

comparative experiment created: sonnet-3.5-c4241dda-vs-gpt-4o-e5ca343e-testcomparison
